In [1]:
import json
import requests
import pandas as pd
import re
import time as tm
from datetime import datetime
from oauth2client.service_account import ServiceAccountCredentials
from gcloud import storage
import numpy as np

In [10]:
with open('running.json','r') as json_file:
    data = json.load(json_file)

In [3]:
cityname = pd.read_csv('citynames.csv',dtype=str)
vilbox = pd.read_csv('vilbox.csv',dtype=str)
info = pd.read_csv('mayornum.csv',encoding="utf-8", dtype="str")
vilname = pd.read_csv('vilname.csv',encoding="utf-8", dtype="str")
counnum = pd.read_csv('councilnum.csv',encoding="utf-8", dtype="str")
counnum['no'] = pd.to_numeric(counnum['no'])
counnum = counnum.sort_values(['no'],ascending=True)
counseat = pd.read_csv('councilseats.csv',encoding="utf-8", dtype="str")
mit = pd.read_csv('mit.csv')
include = []
for i in range(len(mit)):
    s = str(mit['includeTowns'][i]).replace("][",",").replace("'","").strip('][').split(',')
    include.append(s)
mit['includeTowns'] = include

In [11]:
dfC = pd.DataFrame.from_dict(data['TC'],orient='columns')
#整個縣市統整
county = dfC[pd.isnull(dfC['deptCode'])]
county['prvCityArea'] = county['prvCode']+county['cityCode']
county = pd.merge(county, cityname, left_on = 'prvCityArea', right_on = 'fullcode', how='left')
aid = sorted(set(info['fullcode'].tolist()))
info1 = info[['fullcode','area','name','no','party']]
candinfos = []
for i in aid:
    candinfo = info1.loc[info1['fullcode'] == i]
    candinfo = candinfo[['name','no','party']].to_dict(orient='records')
    candinfos.append(candinfo)
infos = pd.DataFrame()
infos['candinfo'] = candinfos
infos['fullcode'] = aid
infos['fullcode'] = infos['fullcode'].astype(str)
county = pd.merge(county, infos, left_on = 'fullcode', right_on = 'fullcode', how='left')
county.set_index('fullcode')
county['process'] = county['tboxRcv'] / county['tboxTot']
county['electionType'] = "city"
tksinfo = []
for i, j in zip(county['candTksInfo'], county['candinfo']):
    t = []
    for k,l in enumerate(i):
        try:
            l['name'] = j[k]['name']
            l['party'] = j[k]['party']
            t.append(l)
        except:
            continue
    for m in range(len(t)):
        t[m]['tickets'] = t[m].pop('tks')
        t[m]['ratio'] = t[m].pop('tksRate')
        t[m]['isYou'] = t[m].pop('candVictor')
    tksinfo.append(t)
county['candTksInfo'] = tksinfo
county.rename(columns = {'fullcode':'code','fullname':'name'}, inplace = True)
countyres = county[['process','name','code','electionType']]
countyres['nowWinParty'] = ''
countyres['candidates'] = ''
for i in countyres.index:
    winParty = pd.DataFrame()
    df = pd.DataFrame()
    v = countyres.loc[i]['name']
    for row in county.loc[county['name'] == v]['candTksInfo']:
        table = json.loads(str(row).replace("'", "\""))
        df1 = pd.DataFrame(table)
        df = pd.concat([df, df1]).sort_values(by = 'tickets', ascending = False)
    df['dif'] = abs(df.nlargest(2, ['ratio'])['ratio'].diff())[1:]
    winParty.loc[i, 'party'] = df.iloc[df.tickets.argmax(), 2:3].item()
    winParty.loc[i, 'ratio']  = df.loc[~pd.isnull(df['dif'])]['dif'].item()
    countyres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
    countyres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
#towns
town = dfC[~pd.isnull(dfC['deptCode'])]
town = town[pd.isnull(dfC['tboxNo'])]
town['prvCityArea'] = town['prvCode']+town['cityCode']+town['deptCode']
town = pd.merge(town, cityname, left_on = 'prvCityArea', right_on = 'fullcode', how='left')
town['countyCode'] = town['fullcode'].str[:5]
info1 = info[['fullcode','area','name','no','party']]
candinfos = []
for i in aid:
    candinfo = info1.loc[info['fullcode'] == i]
    candinfo = candinfo[['name','no','party']].to_dict(orient='records')
    candinfos.append(candinfo)
infos = pd.DataFrame()
infos['candinfo'] = candinfos
infos['fullcode'] = aid
infos['fullcode'] = infos['fullcode'].astype(str)
town = pd.merge(town, infos, left_on = 'countyCode', right_on = 'fullcode', how='left')
town.set_index('countyCode')
tksinfo = []
for i, j in zip(town['candTksInfo'], town['candinfo']):
    s = []
    for k,l in enumerate(i):
        try:
            l['name'] = j[k]['name']
            l['party'] = j[k]['party']
            s.append(l)
        except:
            continue
#     for m in range(len(s)):
#         s[m]['tickets'] = s[m].pop('tks')
#         s[m]['ratio'] = s[m].pop('tksRate')
#         s[m]['isYou'] = s[m].pop('candVictor')
    tksinfo.append(s)
town['candTksInfo'] = tksinfo
town.rename(columns = {'Name': 'name','prvCityArea':'code'}, inplace = True)
townres = town[['countyCode','name','code']]
townres['nowWinParty'] = ''
townres['candidates'] = ''
for i in townres.index:
    winParty = pd.DataFrame()
    df = pd.DataFrame()
    v = townres.loc[i]['code']
    for row in town.loc[town['code'] == v]['candTksInfo']:
        table = json.loads(str(row).replace("'", "\""))
        df1 = pd.DataFrame(table)
        df = pd.concat([df, df1]).sort_values(by = 'tks', ascending = False)
        df.rename(columns = {'tks':'tickets','tksRate':'ratio','candVictor':'isYou'}, inplace = True)
    df['dif'] = abs(df.nlargest(2, ['ratio'])['ratio'].diff())[1:]
    winParty.loc[i, 'party'] = df.iloc[df.tickets.argmax(), 5:6].item()
    winParty.loc[i, 'ratio']  = df.loc[~pd.isnull(df['dif'])]['dif'].item()
    townres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
    townres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
countyres['towns'] = ''
for i in countyres.index:
    c = countyres.loc[i]['code']
    t = townres.loc[townres['countyCode'] == c]
    t = t[['name','code','candidates','nowWinParty']]
    countyres.loc[i, 'towns'] = t.to_json(force_ascii=False, orient='records')
countyres

C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county['prvCityArea'] = county['prvCode']+county['cityCode']
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  countyres['nowWinParty'] = ''
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\2911076669.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

,process,name,code,electionType,nowWinParty,candidates,towns
0,1.0,連江縣,09007,city,"[{""party"":""國民黨"",""ratio"":0.82}]","[{""candNo"":3,""name"":""王忠銘"",""party"":""國民黨"",""ticke...","[{""name"":""南竿鄉"",""code"":""09007010"",""candidates"":..."
1,1.0,金門縣,09020,city,"[{""party"":""無黨籍"",""ratio"":2.37}]","[{""candNo"":3,""name"":""林志錦"",""party"":""無黨籍"",""ticke...","[{""name"":""金城鎮"",""code"":""09020010"",""candidates"":..."
2,1.0,宜蘭縣,10002,city,"[{""party"":""天一黨"",""ratio"":1.97}]","[{""candNo"":2,""name"":""朱振東"",""party"":""天一黨"",""ticke...","[{""name"":""宜蘭市"",""code"":""10002010"",""candidates"":..."
3,1.0,新竹縣,10004,city,"[{""party"":""無黨籍"",""ratio"":0.7}]","[{""candNo"":5,""name"":""劉復嵐"",""party"":""無黨籍"",""ticke...","[{""name"":""竹北市"",""code"":""10004010"",""candidates"":..."
4,1.0,苗栗縣,10005,city,"[{""party"":""無黨籍"",""ratio"":0.39}]","[{""candNo"":1,""name"":""鍾東錦"",""party"":""無黨籍"",""ticke...","[{""name"":""苗栗市"",""code"":""10005010"",""candidates"":..."
5,1.0,彰化縣,10007,city,"[{""party"":""民進黨"",""ratio"":0.78}]","[{""candNo"":2,""name"":""黃秀芳"",""party"":""民進黨"",""ticke...","[{""name"":""彰化市"",""code"":""10007010"",""candidates"":..."
6,1.0,南投縣,10008,city,"[{""party"":""無黨籍"",""ratio"":0.51}]","[{""candNo"":3,""name"":""王永慶"",""party"":""無黨籍"",""ticke...","[{""name"":""南投市"",""code"":""10008010"",""candidates"":..."
7,1.0,雲林縣,10009,city,"[{""party"":""民進黨"",""ratio"":1.29}]","[{""candNo"":2,""name"":""劉建國"",""party"":""民進黨"",""ticke...","[{""name"":""斗六市"",""code"":""10009010"",""candidates"":..."
8,1.0,嘉義縣,10010,city,"[{""party"":""國民黨"",""ratio"":5.2}]","[{""candNo"":1,""name"":""王育敏"",""party"":""國民黨"",""ticke...","[{""name"":""太保市"",""code"":""10010010"",""candidates"":..."
9,1.0,屏東縣,10013,city,"[{""party"":""時代力量"",""ratio"":9.85}]","[{""candNo"":1,""name"":""詹智鈞"",""party"":""時代力量"",""tick...","[{""name"":""屏東市"",""code"":""10013010"",""candidates"":..."


In [ ]:
vil = dfC[~pd.isnull(dfC['tboxNo'])]
vil['tboxNo']  = vil['tboxNo'].astype('str')
vil['boxCode'] = vil['prvCode']+vil['cityCode']+vil['tboxNo']
vil = pd.merge(vil, vilbox, left_on = 'boxCode', right_on = 'fullcode', how='left')
vil['countyCode'] = vil['fullcode'].str[:5]
vil = pd.merge(vil, vilname, left_on = 'fullname', right_on = 'fullname', how='left')
info1 = info[['fullcode','area','name','no','party']]
candinfos = []
for i in aid:
    candinfo = info1.loc[info['fullcode'] == i]
    candinfo = candinfo[['name','no','party']].to_dict(orient='records')
    candinfos.append(candinfo)
infos = pd.DataFrame()
infos['candinfo'] = candinfos
infos['fullcode'] = aid
infos['fullcode'] = infos['fullcode'].astype(str)
vil = pd.merge(vil, infos, left_on = 'countyCode', right_on = 'fullcode', how='left')
vil.set_index('countyCode')
tksinfo = []
for i, j in zip(vil['candTksInfo'], vil['candinfo']):
    t = []
    for k,l in enumerate(i):
        try:
            l['name'] = j[k]['name']
            l['party'] = j[k]['party']
            t.append(l)
        except:
            continue
    tksinfo.append(t)
vil['candTksInfo'] = tksinfo
vil.rename(columns = {'fullcode_y':"townCode",'fullcodename':'code', 'vilname_y': 'name'}, inplace = True)
vilCal = vil[['townCode','name', 'code']].drop_duplicates()
vilCal['candidates'] = ''
vilCal['nowWinParty'] = ''
for i in vilCal.index:
    df = pd.DataFrame()
    winParty = pd.DataFrame()
    v = vilCal.loc[i]['code']
    for row in vil.loc[vil['code'] == v]['candTksInfo']:
        table = json.loads(str(row).replace("'", "\""))
        df1 = pd.DataFrame(table)
        df = pd.concat([df, df1])
    df = pd.pivot_table(df, values='tks', index=['candNo', 'name', 'party'], aggfunc=np.sum).reset_index()
    df['prof1'] = sum(df['tks'])
    df['ratio'] = df['tks']/df['prof1'] 
    df['dif'] = abs(df.nlargest(2, ['ratio'])['ratio'].diff())[1:]
    df.rename(columns = {'tks':'tickets','candVictor':'isYou'}, inplace = True)
    df = df.sort_values(by = 'ratio', ascending = False)
    winParty.loc[i, 'party'] = df.iloc[df.tks.argmax(), 2:3].item()
    winParty.loc[i, 'ratio']  = df.loc[~pd.isnull(df['dif'])]['dif'].item()
    vilCal.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
    vilCal.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
vilCal

In [ ]:
townres['villages'] = ''
for i in townres.index:
    t = townres.loc[i]['code']
    v = vilCal.loc[vilCal['townCode'] == t]
    v = v[['name','code','candidates','nowWinParty']]
    townres.loc[i, 'villages'] = v.to_json(force_ascii=False, orient='records')
countyres['towns'] = ''
for i in countyres.index:
    c = countyres.loc[i]['code']
    t = townres.loc[townres['countyCode'] == c]
    t = t[['name','code','candidates','nowWinParty','villages']]
    countyres.loc[i, 'towns'] = t.to_json(force_ascii=False, orient='records')

In [43]:
dfL = pd.DataFrame.from_dict(data['T1'],orient='columns')
#區域
coun1 = dfL[pd.isnull(dfL['deptCode'])]
coun1['prvCityArea'] = coun1['prvCode']+coun1['cityCode']+coun1['areaCode']
coun1 = pd.merge(coun1, counseat, left_on = 'prvCityArea', right_on = 'regionCode', how='left')
#平原
dfL2 = pd.DataFrame.from_dict(data['T2'],orient='columns')
coun2 = dfL2[pd.isnull(dfL2['deptCode'])]
coun2['prvCityArea'] = coun2['prvCode']+coun2['cityCode']+coun2['areaCode']
coun2 = pd.merge(coun2, counseat, left_on = 'prvCityArea', right_on = 'regionCode', how='left')
#山原
dfL3 = pd.DataFrame.from_dict(data['T3'],orient='columns')
coun3 = dfL3[pd.isnull(dfL3['deptCode'])]
coun3['prvCityArea'] = coun3['prvCode']+coun3['cityCode']+coun3['areaCode']
coun3 = pd.merge(coun3, counseat, left_on = 'prvCityArea', right_on = 'regionCode', how='left')
frames = [coun1, coun2, coun3]
counall = pd.concat(frames)
aid = sorted(set(counnum['regionCode'].tolist()))
council = counnum[['regionCode','regionName','name','no','party']]
candinfos = []
for i in aid:
    candinfo = council.loc[council['regionCode'] == i]
    candinfo = candinfo[['name','no','party']].to_dict(orient='records')
    candinfos.append(candinfo)
infos = pd.DataFrame()
infos['candinfo'] = candinfos
infos['regionCode'] = aid
infos['regionCode'] = infos['regionCode'].astype(str)
counall = pd.merge(counall, infos, left_on = 'regionCode', right_on = 'regionCode', how='left')
counall.set_index('regionCode')
counall['process'] = counall['tboxRcv'] / counall['tboxTot']
tksinfo = []
for i, j in zip(counall['candTksInfo'], counall['candinfo']):
    t = []
    for k,l in enumerate(i):
        try:
            l['name'] = j[k]['name']
            l['party'] = j[k]['party']
            t.append(l)
        except:
            continue
    for m in range(len(t)):
        t[m]['tickets'] = t[m].pop('tks')
        t[m]['ratio'] = t[m].pop('tksRate')
        t[m]['isYou'] = t[m].pop('candVictor')
    tksinfo.append(t)
counall['candTksInfo'] = tksinfo
counall = pd.merge(counall, mit, left_on = 'regionName', right_on = 'regionName', how='left')
counall['electionType'] = "council"
counall = counall[['cityCode_y', 'regionName','regionCode','electionType','includeTowns','process', 'candTksInfo','seats']]
counall.rename(columns = {'cityCode_y':'cityCode', 'regionName':'name','regionCode':'code','candTksInfo':"candidates",'seats':'needSelect'}, inplace = True)
counall['regionCode'] = counall.loc[:, 'code']
councilData = (counall.groupby(['cityCode'])['regionCode','code','name','electionType','includeTowns','process','candidates','needSelect']
               .apply(lambda x: x.set_index('regionCode').to_dict('index')))

C:\Users\apuser\AppData\Local\Temp\ipykernel_20464\156083625.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coun1['prvCityArea'] = coun1['prvCode']+coun1['cityCode']+coun1['areaCode']
C:\Users\apuser\AppData\Local\Temp\ipykernel_20464\156083625.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coun2['prvCityArea'] = coun2['prvCode']+coun2['cityCode']+coun2['areaCode']
C:\Users\apuser\AppData\Local\Temp\ipykernel_20464\156083625.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [12]:
import json
import time
import os
import shutil
from ftplib import FTP
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pandas as pd
from pandas import json_normalize

target_file = 'C:/Users/apuser/Documents/Python Scripts/election/forreal/final.json'

In [ ]:
while os.path.exists(target_file) == False:
    loopbegin = time.time()
    t = time.ctime(loopbegin)
   # print(t+'開始跑囉！')
    with open('C:/Users/apuser/Documents/Python Scripts/election/forreal/running.json') as j:
        data = json.load(j)
    dfC = pd.DataFrame.from_dict(data['TC'],orient='columns')
#整個縣市統整
    county = dfC[pd.isnull(dfC['deptCode'])]
    county['prvCityArea'] = county['prvCode']+county['cityCode']
    county = pd.merge(county, cityname, left_on = 'prvCityArea', right_on = 'fullcode', how='left')
    aid = sorted(set(info['fullcode'].tolist()))
    info1 = info[['fullcode','area','name','no','party']]
    candinfos = []
    for i in aid:
        candinfo = info1.loc[info1['fullcode'] == i]
        candinfo = candinfo[['name','no','party']].to_dict(orient='records')
        candinfos.append(candinfo)
    infos = pd.DataFrame()
    infos['candinfo'] = candinfos
    infos['fullcode'] = aid
    infos['fullcode'] = infos['fullcode'].astype(str)
    county = pd.merge(county, infos, left_on = 'fullcode', right_on = 'fullcode', how='left')
    county.set_index('fullcode')
    county['process'] = county['tboxRcv'] / county['tboxTot']
    county['electionType'] = "city"
    tksinfo = []
    for i, j in zip(county['candTksInfo'], county['candinfo']):
        t = []
        for k,l in enumerate(i):
            try:
                l['name'] = j[k]['name']
                l['party'] = j[k]['party']
                t.append(l)
            except:
                continue
        for m in range(len(t)):
            t[m]['tickets'] = t[m].pop('tks')
            t[m]['ratio'] = t[m].pop('tksRate')
            t[m]['isYou'] = t[m].pop('candVictor')
        tksinfo.append(t)
    county['candTksInfo'] = tksinfo
    county.rename(columns = {'fullcode':'code','fullname':'name'}, inplace = True)
    countyres = county[['process','name','code','electionType']]
    countyres['nowWinParty'] = ''
    countyres['candidates'] = ''
    for i in countyres.index:
        winParty = pd.DataFrame()
        df = pd.DataFrame()
        v = countyres.loc[i]['name']
        for row in county.loc[county['name'] == v]['candTksInfo']:
            table = json.loads(str(row).replace("'", "\""))
            df1 = pd.DataFrame(table)
            df = pd.concat([df, df1]).sort_values(by = 'tickets', ascending = False)
        df['dif'] = abs(df.nlargest(2, ['ratio'])['ratio'].diff())[1:]
        winParty.loc[i, 'party'] = df.iloc[df.tickets.argmax(), 2:3].item()
        winParty.loc[i, 'ratio']  = df.loc[~pd.isnull(df['dif'])]['dif'].item()
        countyres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
        countyres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
#towns
    town = dfC[~pd.isnull(dfC['deptCode'])]
    town = town[pd.isnull(dfC['tboxNo'])]
    town['prvCityArea'] = town['prvCode']+town['cityCode']+town['deptCode']
    town = pd.merge(town, cityname, left_on = 'prvCityArea', right_on = 'fullcode', how='left')
    town['countyCode'] = town['fullcode'].str[:5]
    info1 = info[['fullcode','area','name','no','party']]
    candinfos = []
    for i in aid:
        candinfo = info1.loc[info['fullcode'] == i]
        candinfo = candinfo[['name','no','party']].to_dict(orient='records')
        candinfos.append(candinfo)
    infos = pd.DataFrame()
    infos['candinfo'] = candinfos
    infos['fullcode'] = aid
    infos['fullcode'] = infos['fullcode'].astype(str)
    town = pd.merge(town, infos, left_on = 'countyCode', right_on = 'fullcode', how='left')
    town.set_index('countyCode')
    tksinfo = []
    for i, j in zip(town['candTksInfo'], town['candinfo']):
        s = []
        for k,l in enumerate(i):
            try:
                l['name'] = j[k]['name']
                l['party'] = j[k]['party']
                s.append(l)
            except:
                continue
        tksinfo.append(s)
    town['candTksInfo'] = tksinfo
    town.rename(columns = {'Name': 'name','prvCityArea':'code'}, inplace = True)
    townres = town[['countyCode','name','code']]
    townres['nowWinParty'] = ''
    townres['candidates'] = ''
    for i in townres.index:
        winParty = pd.DataFrame()
        df = pd.DataFrame()
        v = townres.loc[i]['code']
        for row in town.loc[town['code'] == v]['candTksInfo']:
            table = json.loads(str(row).replace("'", "\""))
            df1 = pd.DataFrame(table)
            df = pd.concat([df, df1]).sort_values(by = 'tks', ascending = False)
            df.rename(columns = {'tks':'tickets','tksRate':'ratio','candVictor':'isYou'}, inplace = True)
        df['dif'] = abs(df.nlargest(2, ['ratio'])['ratio'].diff())[1:]
        winParty.loc[i, 'party'] = df.iloc[df.tickets.argmax(), 5:6].item()
        winParty.loc[i, 'ratio']  = df.loc[~pd.isnull(df['dif'])]['dif'].item()
        townres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
        townres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
    countyres['towns'] = ''
    for i in countyres.index:
        c = countyres.loc[i]['code']
        t = townres.loc[townres['countyCode'] == c]
        t = t[['name','code','candidates','nowWinParty']]
        countyres.loc[i, 'towns'] = t.to_json(force_ascii=False, orient='records')
        mayor = countyres.to_json(orient="records", force_ascii=False)
        mayor.replace('\\"',"\"").replace('\\"',"\"").replace('\\"',"\"")
    with open('city.json', 'w', encoding='utf-8') as fp:
        fp.write(mayor)

    file_path = 'city.json'
    with FTP('210.69.89.70', '20220303', 'CHEN2030') as ftp, open(file_path, 'rb') as file:
        ftp.cwd('/2022election-api')
        ftp.storbinary(f'STOR {file_path}', file)
    with FTP('210.69.89.70', '20220303', 'CHEN2030') as ftp, open(file_path, 'rb') as file:
        ftp.cwd('/2022election-api')
        ftp.storbinary(f'STOR {file_path}', file)

#     with FTP('172.17.242.167', '20090948', 'u0401006@CNA123') as ftp, open(file_path, 'rb') as file:
#         ftp.cwd('/2022election-api')
#         ftp.storbinary(f'STOR {file_path}', file)
#     with FTP('172.16.142.167', '20090948', 'u0401006@CNA123') as ftp, open(file_path, 'rb') as file:
#         ftp.cwd('/2022election-api')
#         ftp.storbinary(f'STOR {file_path}', file)

    print(f'{file_path} uploaded!')

    #休息時間
    duration = time.time()- loopbegin
    try:
        print(f'休息{int(180-duration)}秒，還沒結束ㄛ')
        time.sleep(int(180-duration))
    except:
        continue

C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county['prvCityArea'] = county['prvCode']+county['cityCode']
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  countyres['nowWinParty'] = ''
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'nowWinParty'] = winParty.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  townres.loc[i, 'candidates'] = df.to_json(force_ascii=False, orient='records')
C:\Users\apuser\AppData\Local\Temp\ipykernel_5504\1610727104.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

city.json uploaded!
休息178秒，還沒結束ㄛ


In [18]:
mayor.replace('\\"',"\"").replace('\\"',"\"").replace('\\"',"\"")

'[{"process":1.0,"name":"連江縣","code":"09007","electionType":"city","nowWinParty":"[{"party":"國民黨","ratio":4.47}]","candidates":"[{"candNo":2,"name":"曹爾元","party":"國民黨","tickets":1871,"ratio":27.0,"isYou":" ","dif":null},{"candNo":3,"name":"王忠銘","party":"國民黨","tickets":1561,"ratio":22.53,"isYou":" ","dif":4.47},{"candNo":1,"name":"李問","party":"民進黨","tickets":1249,"ratio":18.03,"isYou":" ","dif":null}]","towns":"[{"name":"南竿鄉","code":"09007010","candidates":"[{"candNo":2,"tickets":1297,"isYou":" ","ratio":27.37,"name":"曹爾元","party":"國民黨","dif":null},{"candNo":3,"tickets":943,"isYou":" ","ratio":19.9,"name":"王忠銘","party":"國民黨","dif":7.47},{"candNo":1,"tickets":804,"isYou":" ","ratio":16.97,"name":"李問","party":"民進黨","dif":null}]","nowWinParty":"[{"party":"國民黨","ratio":7.47}]"},{"name":"北竿鄉","code":"09007020","candidates":"[{"candNo":2,"tickets":278,"isYou":" ","ratio":27.55,"name":"曹爾元","party":"國民黨","dif":null},{"candNo":3,"tickets":252,"isYou":" ","ratio":24.98,"name":"王忠銘","party":"國民黨"

In [100]:
while os.path.exists(target_file) == False:
    loopbegin = time.time()
    t = time.ctime(loopbegin)
    print(t+'開始跑囉！')
    with open('C:/Users/apuser/Documents/Python Scripts/election/forreal/running.json') as j:
        data = json.load(j)
    dfL = pd.DataFrame.from_dict(data['T1'],orient='columns')
#區域
    coun1 = dfL[pd.isnull(dfL['deptCode'])]
    coun1['prvCityArea'] = coun1['prvCode']+coun1['cityCode']+coun1['areaCode']
    coun1 = pd.merge(coun1, counseat, left_on = 'prvCityArea', right_on = 'regionCode', how='left')
#平原
    dfL2 = pd.DataFrame.from_dict(data['T2'],orient='columns')
    coun2 = dfL2[pd.isnull(dfL2['deptCode'])]
    coun2['prvCityArea'] = coun2['prvCode']+coun2['cityCode']+coun2['areaCode']
    coun2 = pd.merge(coun2, counseat, left_on = 'prvCityArea', right_on = 'regionCode', how='left')
#山原
    dfL3 = pd.DataFrame.from_dict(data['T3'],orient='columns')
    coun3 = dfL3[pd.isnull(dfL3['deptCode'])]
    coun3['prvCityArea'] = coun3['prvCode']+coun3['cityCode']+coun3['areaCode']
    coun3 = pd.merge(coun3, counseat, left_on = 'prvCityArea', right_on = 'regionCode', how='left')
    frames = [coun1, coun2, coun3]
    counall = pd.concat(frames)
    aid = sorted(set(counnum['regionCode'].tolist()))
    council = counnum[['regionCode','regionName','name','no','party']]
    candinfos = []
    for i in aid:
        candinfo = council.loc[council['regionCode'] == i]
        candinfo = candinfo[['name','no','party']].to_dict(orient='records')
        candinfos.append(candinfo)
    infos = pd.DataFrame()
    infos['candinfo'] = candinfos
    infos['regionCode'] = aid
    infos['regionCode'] = infos['regionCode'].astype(str)
    counall = pd.merge(counall, infos, left_on = 'regionCode', right_on = 'regionCode', how='left')
    counall.set_index('regionCode')
    counall.set_index('regionCode')
    counall['process'] = counall['tboxRcv'] / counall['tboxTot']
    tksinfo = []
    for i, j in zip(counall['candTksInfo'], counall['candinfo']):
        t = []
        for k,l in enumerate(i):
            try:
                l['name'] = j[k]['name']
                l['party'] = j[k]['party']
                t.append(l)
            except:
                continue
        for m in range(len(t)):
            t[m]['tickets'] = t[m].pop('tks')
            t[m]['ratio'] = t[m].pop('tksRate')
            t[m]['isYou'] = t[m].pop('candVictor')
        tksinfo.append(t)
    counall['candTksInfo'] = tksinfo
    counall = pd.merge(counall, mit, left_on = 'regionName', right_on = 'regionName', how='left')
    counall['electionType'] = "council"
    counall = counall[['cityCode_y', 'regionName','regionCode','electionType','includeTowns','process', 'candTksInfo','seats']]
    counall.rename(columns = {'cityCode_y':'cityCode', 'regionName':'name','regionCode':'code','candTksInfo':"candidates",'seats':'needSelect'}, inplace = True)
    counall['regionCode'] = counall.loc[:, 'code']
    councilData = (counall.groupby(['cityCode'])['regionCode','code','name','electionType','includeTowns','process','candidates','needSelect']
                   .apply(lambda x: x.set_index('regionCode').to_dict('index'))
                   .to_json(force_ascii=False))
    councilData.replace('\\"',"\"")
    with open('council.json', 'w', encoding='utf-8') as fp:
        fp.write(councilData)
    file_path = 'council.json'
    with FTP('210.69.89.70', '20220303', 'CHEN2030') as ftp, open(file_path, 'rb') as file:
        ftp.cwd('/2022election-api')
        ftp.storbinary(f'STOR {file_path}', file)
    with FTP('210.69.89.70', '20220303', 'CHEN2030') as ftp, open(file_path, 'rb') as file:
        ftp.cwd('/2022election-api')
        ftp.storbinary(f'STOR {file_path}', file)

#     with FTP('172.17.242.167', '20090948', 'u0401006@CNA123') as ftp, open(file_path, 'rb') as file:
#         ftp.cwd('/2022election-api')
#         ftp.storbinary(f'STOR {file_path}', file)
#     with FTP('172.16.142.167', '20090948', 'u0401006@CNA123') as ftp, open(file_path, 'rb') as file:
#         ftp.cwd('/2022election-api')
#         ftp.storbinary(f'STOR {file_path}', file)

    print(f'{file_path} uploaded!')

    #休息時間
    duration = time.time()- loopbegin
    try:
        print(f'休息{int(180-duration)}秒，還沒結束ㄛ')
        time.sleep(int(180-duration))
    except:
        continue

Wed Nov 16 18:56:13 2022開始跑囉！


C:\Users\apuser\AppData\Local\Temp\ipykernel_14812\328745215.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coun1['prvCityArea'] = coun1['prvCode']+coun1['cityCode']+coun1['areaCode']
C:\Users\apuser\AppData\Local\Temp\ipykernel_14812\328745215.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coun2['prvCityArea'] = coun2['prvCode']+coun2['cityCode']+coun2['areaCode']
C:\Users\apuser\AppData\Local\Temp\ipykernel_14812\328745215.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy o

council.json uploaded!
休息179秒，還沒結束ㄛ


In [43]:
#整理投票所以及對應的村里
vilname = pd.read_csv('vilname.csv',dtype=str)
dfV = pd.DataFrame.from_dict(data['TV'],orient='columns')
dfV = dfV[~pd.isnull(dfV['tboxNo'])]
dfV['num'] = [len(i) for i in dfV['candTksInfo']]
dfV['prvCityArea'] = dfV['prvCode']+dfV['cityCode']+dfV['deptCode']+dfV['liCode']
dfV = dfV.loc[dfV['num']> 0]
dfV['prog'] = dfV['tboxRcv'] / dfV['tboxTot']
dfV = pd.merge(dfV, vilname, left_on = 'prvCityArea', right_on = 'fullcodename', how='left')
dfV = dfV[['prvCode','cityCode','deptCode','tboxNo','fullname','num','candTksInfo','prof1','prog']]
dfV['tboxNo']  = dfV['tboxNo'].astype('str')

,prvCode,cityCode,deptCode,tboxNo,fullname,num,candTksInfo,prof1,prog
0,09,007,010,1.0,連江縣南竿鄉介壽村,3,"[{'candNo': 1, 'tks': 103, 'candVictor': ' ', ...",329,1.0
1,09,007,010,2.0,連江縣南竿鄉介壽村,3,"[{'candNo': 1, 'tks': 80, 'candVictor': ' ', '...",439,1.0
2,09,007,010,3.0,連江縣南竿鄉復興村,3,"[{'candNo': 1, 'tks': 211, 'candVictor': '*', ...",368,1.0
3,09,007,010,3.0,連江縣南竿鄉福沃村,1,"[{'candNo': 1, 'tks': 415, 'candVictor': '*', ...",415,1.0
4,09,007,010,4.0,連江縣南竿鄉清水村,1,"[{'candNo': 1, 'tks': 431, 'candVictor': '*', ...",431,1.0
...,...,...,...,...,...,...,...,...,...
17657,68,000,130,708.0,桃園市復興區三光里,2,"[{'candNo': 1, 'tks': 196, 'candVictor': '*', ...",373,1.0
17658,68,000,130,706.0,桃園市復興區奎輝里,2,"[{'candNo': 1, 'tks': 63, 'candVictor': ' ', '...",417,1.0
17659,68,000,130,709.0,桃園市復興區華陵里,2,"[{'candNo': 1, 'tks': 95, 'candVictor': ' ', '...",598,1.0
17660,68,000,130,704.0,桃園市復興區羅浮里,4,"[{'candNo': 1, 'tks': 184, 'candVictor': '*', ...",550,1.0


In [44]:
dfV.to_csv('vilbox.csv')

In [15]:
#整理行政區以及對應的選區
mtt = mt[['townCode', 'regionCode']]
mtt = mtt.merge(mtt.groupby("townCode", as_index=False).agg(list)\
           .rename(columns={'regionCode':'allRegions'}))
mtt = mtt[['townCode', 'allRegions']].drop_duplicates(subset = ["townCode"])
townRegionMap = mtt.to_json(orient = 'records')
with open("townRegionMap.json", "w") as outfile:
    json.dump(townRegionMap, outfile)

In [30]:
#續上
mt = pd.read_csv('match.csv',dtype=str)
mit = mt[["regionName","Towns"]]
mit = mit.merge(mit.groupby("regionName", as_index=False).agg(list)\
           .rename(columns={'Towns':'includeTowns'}))
mit = mit[['regionName', 'includeTowns']]
mit = mit.drop_duplicates(subset = ["regionName"])
mit.to_csv('mit.csv')